## Cockcroft - Gault ( Peso magro o peso total )

In [1]:
DECIMALS = 3


def get_peso_magro(genero, peso, imc):
    """ Toma el peso magro """
    if genero == "masculino":
        ind1 = 6680
        ind2 = 216
    else:
        ind1 = 6780
        ind2 = 244
    formula = (9270 * peso) / (ind1 + (ind2 * imc))
    return round(formula, DECIMALS)


def get_imc(peso, talla):
    """ Calcula IMC """
    return round(peso / ((talla * talla) / 10000), DECIMALS)

def get_superficie_corp(peso, talla):
    """ Get superficie corporal """
    return (peso**0.425)*(talla**0.725)*0.007184


def get_cg_clearance_creatinina(edad, peso, talla, genero, creatinina):
    """ Toma el clearence creatinina """
    imc = get_imc(peso, talla)
    _peso = None
    
    multi_genero = 1 if genero == "masculino" else 0.85
    
    if imc > 30:
        _peso = get_peso_magro(genero, peso, imc)
    else:
        _peso = peso
    total = ((140 - edad) * _peso / (72 * creatinina)) * multi_genero
    return round(total, 1)


def get_cg_estimacion_vfg(edad, peso, talla, genero, creatinina):
    """ Get estimación VFG Cockgroft Gault """
    superficie_corp = get_superficie_corp(peso, talla)
    clear_creatinina = get_cg_clearance_creatinina(edad, peso, talla, genero, creatinina)
    return round(( 1.73 / superficie_corp ) * clear_creatinina, 1)

In [2]:
edad = 57
peso = 120
talla = 180
genero = "masculino"
creatinina = 0.26

In [3]:
clear_creatinina = get_cg_clearance_creatinina(edad, peso, talla, genero, creatinina)

In [4]:
estimacion_vfg = get_cg_estimacion_vfg(edad, peso, talla, genero, creatinina)

In [5]:
estimacion_vfg

245.1

## CDK-EPICreatinina (2021)

In [6]:
def get_cdk_epicreatinina_2021(creatinina, genero, edad):
    """ Toma el valor de la formula CDK-EPICreatinina (2021) """
    if genero == "masculino":
        var_genero_a = 0.9
        multi_genero = 1
        if creatinina <= 0.9:
            var_genero_b = -0.302
        else:
            var_genero_b = -1.2
    else:
        var_genero_a = 0.7
        multi_genero = 1.012
        if creatinina <= 0.7:
            var_genero_b = -0.241
        else:
            var_genero_b = -1.2
    formula = 142 * ((creatinina/var_genero_a) ** var_genero_b) * (0.9938**edad) * multi_genero
    return round(formula, 1)

def get_vfg_no_normalizada(creatinina, genero, edad, peso, talla):
    """ Toma la VFG no normalizada de la función CKD-EPIcreatinina_2021 """
    cdk_epicreatinina_2021 = get_cdk_epicreatinina_2021(creatinina, genero, edad)
    superficie_corp = get_superficie_corp(peso, talla)
    formula = (superficie_corp/1.73)*cdk_epicreatinina_2021
    return round(formula, 1)

In [7]:
creatinina2 = 0.26
genero2 = "masculino"
edad2 = 57
peso2 = 93
talla2 = 180
get_vfg_no_normalizada(creatinina2, genero2, edad2, peso2, talla2)

178.3

## CKD-EPIcrea-cys (2021)

In [8]:
def get_ckd_epicrea_cys_2021(creatinina, cistanina, genero, edad):
    multi_genero = 1 if genero == "masculino" else 0.963
    if genero == "masculino":
        if creatinina <= 0.9:
            if cistanina <= 0.8:
                A = 0.9
                B = -0.144
                C = 0.8
                B = -0.323
            else:
                A = 0.9
                B = -0.144
                C = 0.8
                D = -0.778
        else:
            if cistanina <= 0.8:
                A = 0.9
                B = -0.544
                C = 0.8
                D = -0.323
            else:
                A = 0.9
                B = -0.544
                C = 0.8
                D = -0.778
    else:
        if creatinina <= 0.7:
            if cistanina <= 0.8:
                A = 0.7
                B = -0.219
                C = 0.8
                D =-0.323
            else:
                A = 0.7
                B = -0.219
                C = 0.8
                D =-0.778
        else:
            if cistanina <= 0.8:
                A = 0.7
                B = -0.544
                C = 0.8
                D = -0.323
            else:
                A = 0.7
                B = -0.544
                C = 0.8
                D = -0.778    
    formula = 135 * ((creatinina/A)**B) * ((cistanina/C)**D) * (0.9961 ** edad) * multi_genero
    return round(formula, 1)


def get_ckd_epicrea_cys_vfg_no_norm(creatinina, cistanina, genero, edad, peso, talla):
    """ CKD-EPIcrea-cys (2021) VFG no normalizada """
    estimacion_vfg = get_ckd_epicrea_cys_2021(creatinina, cistanina, genero, edad)
    superficie_corp = get_superficie_corp(peso, talla)
    formula = (superficie_corp / 1.73) * estimacion_vfg
    return round(formula, 1)

In [9]:
creatinina2 = 0.26
genero2 = "masculino"
edad2 = 57
peso2 = 93
talla2 = 180
cistanina2 = 1.03
get_ckd_epicrea_cys_vfg_no_norm(creatinina2, cistanina2, genero2, edad2, peso2, talla2)

130.5

In [10]:
get_ckd_epicrea_cys_2021(creatinina2, cistanina2, genero2, edad2)

106.1

## BIS 1

In [16]:
def bis1(creatinina, edad, genero):
    """ VFG BIS 1"""
    multi_genero = 1 if genero == "masculino" else 0.82
    formula = 3736 * (creatinina**-0.87) * (edad**-0.95) * multi_genero
    return round(formula, 1)

In [17]:
bis1(creatinina2, edad2, genero2)

259.0

In [18]:
def bis2(cistatina, creatinina, edad, genero):
    """ VFG BIS2 """
    multi_genero = 1 if genero == "masculino" else 0.87
    formula = 767 * (cistatina**-0.61) * (creatinina**-0.40) * (edad**-0.57) * multi_genero
    return round(formula, 1)

In [19]:
bis2(cistanina2, creatinina2, edad2, genero2)

128.9